<a href="https://colab.research.google.com/github/albertofernandezvillan/computer-vision-and-deep-learning-course/blob/main/configure_opencv_with_gpu_on_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" style="padding-right:10px;" src ="https://raw.githubusercontent.com/albertofernandezvillan/computer-vision-and-deep-learning-course/main/assets/university_oviedo_logo.png" width=300 px>

This notebook is from the Course "***Computer vision in the new era of Artificial Intelligence and Deep Learning***", or "*Visión por computador en la nueva era de la Inteligencia Artificial y el Deep Learning*" (ES) from the "Second quarter university extension courses" that the University of Oviedo is offering (05/04/2021 - 16/04/2021)

<[Github Repository](https://github.com/albertofernandezvillan/computer-vision-and-deep-learning-course) | [Course Web Page Information](https://www.uniovi.es/estudios/extension/cursos2c/-/asset_publisher/SEp0PJi4ISGo/content/vision-por-computador-en-la-nueva-era-de-la-inteligencia-artificial-y-el-deep-learning?redirect=%2Festudios%2Fextension%2Fcursos2c)>

# Summary

In this notebook, you will see how to install and configure OpenCV in Colab with GPU support. **OpenCV 4.2 now supports NVIDIA GPUs for inference using OpenCV’s dnn module, improving inference speed**. This notebook is based in this [wonderful tutorial](https://www.pyimagesearch.com/2020/02/03/how-to-use-opencvs-dnn-module-with-nvidia-gpus-cuda-and-cudnn/), which describes how to install and compile OpenCV with GPU support. Installing OpenCV for GPU support is easier in Colab because this environment has many dependencies onboard. Indeed this tutorial in more based in the response *berak* gives to the question "[how to make openCV use GPU on google colab](https://answers.opencv.org/question/233476/how-to-make-opencv-use-gpu-on-google-colab/)". 

At the time of writing this tutorial, the current OpenCV installation is '4.1.2'. Moreover, despite the fact that the version would be >= 4.2, some flags should be used to compile the library. **Using OpenCV’s dnn module requires you to compile OpenCV from source code: you cannot "pip install" OpenCV with GPU support**.
So, if you install OpenCV via pip (`pip install opencv-python==4.2`) and check these flags (you will see how to check the properties of an OpenCV installation) you will see that they are not included in the installation, and hence, you cannot take advantage of the GPU support for inference using OpenCV's dnn module.

So in this notebook you will see how to:
*   Check the current OpenCV version, install a new version (via pip) and see the 'build information' of the current installation.
*   Configure and install OpenCV with GPU support. This will take some hours to complete.
*   Save the new compiled OpenCV library to be used whenever you want. You should take into account that the new installed OpenCV library is not persistent (next Colab allocation cleans everything). We will save it in Google Drive and also include the option to download the file to your hard disk.
*   Test the installation. Here we have included a public link to download the compiled OpenCV version with GPU support, so you can just jump to the final section of this notebook.

**You can also download the OpenCV 4.5.1 library compiled with GPU support** [from this link](https://drive.google.com/u/0/uc?export=download&confirm=S15D&id=1-Ze3zkdzA_kDsakY_hGAZRh3aK3p5lHk). See the last section of this notebook to see how to install it and test that the installation was correct.


# Introduction

First point is to see the current version of the OpenCV version Colab uses. At the time of writing this tutorial, OpenCV version is '4.1.2'. Moreover and introduced in the previous section, OpenCV 4.2 now supports NVIDIA GPUs for inference using OpenCV’s dnn module, improving inference speed. To configure dnn module (from your Python code) for GPU support, these two lines of code used be included (after a model is loaded and before inference is performed).

```
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)
```

In connection with this, we will benchmark popular deep learning models (pre-trained and ready to use using OpenCV) for both CPU and GPU inference speed and see how to include these aforementioned two lines in your Python scripts and notebooks.

Therefore, once we have installed a OpenCV version > 4.2 from source code, a first step is to see if these attributes (cv2.dnn.DNN_BACKEND_CUDA, cv2.dnn.DNN_TARGET_CUDA) are included. See next how to do it.




In [ ]:
import cv2

print('You need OpenCV 4.2 or above to use DNN_BACKEND_CUDA & DNN_TARGET_CUDA')
print('Lets check it')
print("Current OpenCV installation: '{}'".format(cv2.__version__))
try:
  print("cv2.dnn.DNN_BACKEND_CUDA: '{}'".format(cv2.dnn.DNN_BACKEND_CUDA))
  print("cv2.dnn.DNN_TARGET_CUDA: '{}'".format(cv2.dnn.DNN_TARGET_CUDA))
except AttributeError: 
  print("It seems like your current OpenCV version is < 4.2 with no GPU support")

You need OpenCV 4.2 or above to use DNN_BACKEND_CUDA & DNN_TARGET_CUDA
Lets check it
Current OpenCV installation: '4.1.2'
It seems like your current OpenCV version is < 4.2 with no GPU support


As commented above, using OpenCV’s dnn module requires you to compile OpenCV from source code: you cannot "pip install" OpenCV with GPU support. To check this we can explore how the current installation of OpenCV was built using `cv2.getBuildInformation()`. So if we print the output, we can look for CUDA and related GPU information.

In [ ]:
print(cv2.getBuildInformation())


General configuration for OpenCV 4.1.2 =====================================
  Version control:               4.1.2

  Extra modules:
    Location (extra):            /io/opencv_contrib/modules
    Version control (extra):     4.1.2

  Platform:
    Timestamp:                   2019-11-22T00:43:29Z
    Host:                        Linux 4.15.0-1028-gcp x86_64
    CMake:                       3.9.0
    CMake generator:             Unix Makefiles
    CMake build tool:            /usr/bin/gmake
    Configuration:               Release

  CPU/HW features:
    Baseline:                    SSE SSE2 SSE3
      requested:                 SSE3
    Dispatched code generation:  SSE4_1 SSE4_2 FP16 AVX
      requested:                 SSE4_1 SSE4_2 AVX FP16 AVX2 AVX512_SKX
      SSE4_1 (14 files):         + SSSE3 SSE4_1
      SSE4_2 (1 files):          + SSSE3 SSE4_1 POPCNT SSE4_2
      FP16 (0 files):            + SSSE3 SSE4_1 POPCNT SSE4_2 FP16 AVX
      AVX (4 files):             + SSSE3 SSE4_1

As you could see, the information about GPU, CUDA and related terms are missing. In connetion with this, [You can add whatever support to want to regex search in this to pretty-fy the outputs](https://stackoverflow.com/questions/17347308/how-to-check-if-opencv-was-compiled-with-tbb-cuda-or-qt-support), instead of searching for it in the build-info outputs.

In [ ]:
import cv2
import re

cv_info = [re.sub('\s+', ' ', ci.strip()) for ci in cv2.getBuildInformation().strip().split('\n') 
               if len(ci) > 0 and re.search(r'(nvidia*:?)|(cuda*:)|(cudnn*:)', ci.lower()) is not None]
print(cv_info)

[]


Therefore, we have to install OpenCV from source code for GPU support. See next section to see how to do it.

# Install OpenCV from source code for GPU support

To build OpenCV we will make use of cmake. In this case, the important flags are `-DWITH_CUDA=ON -DWITH_CUBLAS=ON -DWITH_CUDNN=ON -DOPENCV_DNN_CUDA=ON`, which are related to CUDA.
[See here for more information about this](https://answers.opencv.org/question/233476/how-to-make-opencv-use-gpu-on-google-colab/).

At the end of this process (several hours are required) file `cv2.cpython-36m-x86_64-linux-gnu.so` is installed and created. This is the file we will need to save for future installations of OpenCV with GPU support in Colab. 

In [ ]:
%cd /content
!git clone https://github.com/opencv/opencv
!git clone https://github.com/opencv/opencv_contrib
!mkdir /content/build
%cd /content/build
!cmake -DOPENCV_EXTRA_MODULES_PATH=/content/opencv_contrib/modules  -DBUILD_SHARED_LIBS=OFF  -DBUILD_TESTS=OFF  -DBUILD_PERF_TESTS=OFF -DBUILD_EXAMPLES=OFF -DWITH_OPENEXR=OFF -DWITH_CUDA=ON -DWITH_CUBLAS=ON -DWITH_CUDNN=ON -DOPENCV_DNN_CUDA=ON /content/opencv
!make -j8 install


/content
Cloning into 'opencv'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 287455 (delta 2), reused 1 (delta 0), pack-reused 287441
Receiving objects: 100% (287455/287455), 476.87 MiB | 30.02 MiB/s, done.
Resolving deltas: 100% (200845/200845), done.
Checking out files: 100% (6727/6727), done.
Cloning into 'opencv_contrib'...
remote: Enumerating objects: 34593, done.
remote: Total 34593 (delta 0), reused 0 (delta 0), pack-reused 34593
Receiving objects: 100% (34593/34593), 130.43 MiB | 30.59 MiB/s, done.
Resolving deltas: 100% (21409/21409), done.
/content/build
-- The CXX compiler identification is GNU 7.5.0
-- The C compiler identification is GNU 7.5.0
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting C

Now, if you check the installed OpenCV version, we can see that the version is '4.5.1'. Before doing it, maybe we have to restart the runtime in order to "refresh" the installed version of OpenCV.

In [ ]:
# Restart runtime:
import os

def restart_runtime():
  os.kill(os.getpid(), 9)

restart_runtime()

In [ ]:
import cv2

print(cv2.getBuildInformation())


General configuration for OpenCV 4.5.1-dev =====================================
  Version control:               4.5.1-218-gad66b070a7

  Extra modules:
    Location (extra):            /content/opencv_contrib/modules
    Version control (extra):     4.5.1-60-g4e85f8c6

  Platform:
    Timestamp:                   2021-02-15T14:53:42Z
    Host:                        Linux 4.19.112+ x86_64
    CMake:                       3.12.0
    CMake generator:             Unix Makefiles
    CMake build tool:            /usr/bin/make
    Configuration:               Release

  CPU/HW features:
    Baseline:                    SSE SSE2 SSE3
      requested:                 SSE3
    Dispatched code generation:  SSE4_1 SSE4_2 FP16 AVX AVX2 AVX512_SKX
      requested:                 SSE4_1 SSE4_2 AVX FP16 AVX2 AVX512_SKX
      SSE4_1 (15 files):         + SSSE3 SSE4_1
      SSE4_2 (1 files):          + SSSE3 SSE4_1 POPCNT SSE4_2
      FP16 (0 files):            + SSSE3 SSE4_1 POPCNT SSE4_2 FP16 AVX

In [ ]:
import cv2
import re

cv_info = [re.sub('\s+', ' ', ci.strip()) for ci in cv2.getBuildInformation().strip().split('\n') 
               if len(ci) > 0 and re.search(r'(nvidia*:?)|(cuda*:)|(cudnn*:)', ci.lower()) is not None]
print(cv_info)

['NVIDIA CUDA: YES (ver 10.1, CUFFT CUBLAS)', 'NVIDIA GPU arch: 30 35 37 50 52 60 61 70 75', 'NVIDIA PTX archs:', 'cuDNN: YES (ver 7.6.5)']


In [ ]:
import cv2

print('You need OpenCV 4.2 or above to use DNN_BACKEND_CUDA & DNN_TARGET_CUDA')
print('Lets check it')
print("Current OpenCV installation: '{}'".format(cv2.__version__))
try:
  print("cv2.dnn.DNN_BACKEND_CUDA: '{}'".format(cv2.dnn.DNN_BACKEND_CUDA))
  print("cv2.dnn.DNN_TARGET_CUDA: '{}'".format(cv2.dnn.DNN_TARGET_CUDA))
except AttributeError: 
  print("It seems like your current OpenCV version is < 4.2 with no GPU support")

You need OpenCV 4.2 or above to use DNN_BACKEND_CUDA & DNN_TARGET_CUDA
Lets check it
Current OpenCV installation: '4.5.1-dev'
cv2.dnn.DNN_BACKEND_CUDA: '5'
cv2.dnn.DNN_TARGET_CUDA: '6'


At this point, we can: a) save the created file in Google Drive, b) download the file to our hard disk. Both options can also be performed as pleased, and are included as follows.

First, we will see how to save the file in Google Drive. We have to mount Gogle Drive, and then copy the file to a destination folder.

In [ ]:
# Mount Google Drive in Colab:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
# Copy the created file to the desired destination folder:
!cd /content/drive
!mkdir /content/drive/MyDrive/cv2_gpu
!cp /content/build/lib/python3/cv2.cpython-36m-x86_64-linux-gnu.so /content/drive/MyDrive/cv2_gpu

In [ ]:
# Check that the file is correctly copied:
%cd /content/drive/MyDrive/cv2_gpu
!ls -la

/content/drive/MyDrive/cv2_gpu
total 960999
-rw------- 1 root root 984062496 Feb 15 17:59 cv2.cpython-36m-x86_64-linux-gnu.so


Moreover, we can also download the file:

In [ ]:
from google.colab import files

files.download("/content/build/lib/python3/cv2.cpython-36m-x86_64-linux-gnu.so")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Install and test OpenCV with GPU support using the generated file

At this point we can test if the generated OpenCV library is OK. **Notice that this is the only step we have to perform if you want to configure OpenCV with GPU support in your next notebooks**.

What we have to do is to copy the generated file to Colab and restart the runtime to update the OpenCV library. Then, we test that everything is correct.

You can also download the OpenCV 4.5.1 library compiled with GPU support [from this link](https://drive.google.com/u/0/uc?export=download&confirm=S15D&id=1-Ze3zkdzA_kDsakY_hGAZRh3aK3p5lHk), which is available for you in case you want it. If you want to download it from a notebook, you can make use of `gdown`. See next how to do it. 

Notice that after the library is copied into Colab, you should restart the runtime to update the OpenCV library.

Therefore, the first step is to download our OpenCV 4.5.1 library compiled with GPU support.

In [ ]:
# Download "our" OpenCV 4.5.1 library compiled with GPU support
!gdown --id 1-Ze3zkdzA_kDsakY_hGAZRh3aK3p5lHk

Downloading...
From: https://drive.google.com/uc?id=1-Ze3zkdzA_kDsakY_hGAZRh3aK3p5lHk
To: /content/cv2.cpython-36m-x86_64-linux-gnu.so
984MB [00:07, 135MB/s]


We check that the file 'cv2.cpython-36m-x86_64-linux-gnu.so' exists.

In [ ]:
!ls -la

total 961020
drwxr-xr-x 1 root root      4096 Mar 29 21:11 .
drwxr-xr-x 1 root root      4096 Mar 29 20:27 ..
drwxr-xr-x 4 root root      4096 Mar 18 13:36 .config
-rw-r--r-- 1 root root 984062496 Mar 29 21:11 cv2.cpython-36m-x86_64-linux-gnu.so
drwxr-xr-x 1 root root      4096 Mar 18 13:36 sample_data


As second step, we have also to rename the file to cv2.so (seen here: https://gist.github.com/Mahedi-61/804a663b449e4cdb31b5fea96bb9d561)

In [ ]:
!mv cv2.cpython-36m-x86_64-linux-gnu.so cv2.so

In [ ]:
!ls -la

total 961020
drwxr-xr-x 1 root root      4096 Mar 29 21:12 .
drwxr-xr-x 1 root root      4096 Mar 29 20:27 ..
drwxr-xr-x 4 root root      4096 Mar 18 13:36 .config
-rw-r--r-- 1 root root 984062496 Mar 29 21:11 cv2.so
drwxr-xr-x 1 root root      4096 Mar 18 13:36 sample_data


As third and final step, we have to restart the runtime.

In [ ]:
# Restart the runtime:
import os

def restart_runtime():
  os.kill(os.getpid(), 9)

restart_runtime()

At this point, we check the properties of the installed version and verify that everything is OK.

In [ ]:
# Test everything:
import cv2
import re

cv_info = [re.sub('\s+', ' ', ci.strip()) for ci in cv2.getBuildInformation().strip().split('\n') 
               if len(ci) > 0 and re.search(r'(nvidia*:?)|(cuda*:)|(cudnn*:)', ci.lower()) is not None]
print(cv_info)

import cv2

print('You need OpenCV 4.2 or above to use DNN_BACKEND_CUDA & DNN_TARGET_CUDA')
print('Lets check it')
print("Current OpenCV installation: '{}'".format(cv2.__version__))
try:
  print("cv2.dnn.DNN_BACKEND_CUDA: '{}'".format(cv2.dnn.DNN_BACKEND_CUDA))
  print("cv2.dnn.DNN_TARGET_CUDA: '{}'".format(cv2.dnn.DNN_TARGET_CUDA))
except AttributeError: 
  print("It seems like your current OpenCV version is < 4.2 with no ")

['NVIDIA CUDA: YES (ver 10.1, CUFFT CUBLAS)', 'NVIDIA GPU arch: 30 35 37 50 52 60 61 70 75', 'NVIDIA PTX archs:', 'cuDNN: YES (ver 7.6.5)']
You need OpenCV 4.2 or above to use DNN_BACKEND_CUDA & DNN_TARGET_CUDA
Lets check it
Current OpenCV installation: '4.5.1-dev'
cv2.dnn.DNN_BACKEND_CUDA: '5'
cv2.dnn.DNN_TARGET_CUDA: '6'


# Conclusions

OpenCV 4.2 now supports NVIDIA GPUs for inference using OpenCV’s dnn module, improving inference speed. In this notebook we have seen how to install OpenCV with GPU support from source code because OpenCV’s dnn module requires you to compile OpenCV from source code and you cannot "pip install" OpenCV with GPU support.

For your convenience, you can also download the OpenCV 4.5.1 library compiled with GPU support [from this link](https://drive.google.com/u/0/uc?export=download&confirm=S15D&id=1-Ze3zkdzA_kDsakY_hGAZRh3aK3p5lHk). If you want to download it in the notebooks, you can make use of `gdown`:

```
!gdown --id 1-Ze3zkdzA_kDsakY_hGAZRh3aK3p5lHk
```

Finally, in other notebooks, we will benchmark popular deep learning models (pre-trained and ready to use using OpenCV) for both CPU and GPU inference speed and see how to include these aforementioned two lines in your Python scripts and notebooks:

```
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)
```
